# Dataset Preparation for First ICU Prediction

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import pandas as pd
import spacy

from pathlib import Path

In [3]:
path = Path('./data')

## Functions

In [4]:
def get_sample(df, sample_pct=0.01, with_val=True, seed=None):
  train = df.loc[(df['split']) == 'train'].sample(frac=sample_pct, random_state=seed)
  train.reset_index(inplace=True, drop=True)

  if with_val:
    val = df.loc[(df['split']) == 'val'].sample(frac=sample_pct, random_state=seed)
    val.reset_index(inplace=True, drop=True)
    return pd.concat([train, val], axis=0) 

  return train

## Dataset Processing

1. Load in the data
2. Drop duplicates
3. Merge `category`, `description`, and `text` into a new column called `note`
4. Tokenize text using `scispacy` and create new column called `scispacy_note` to save tokenized text
5. Save the relevant columns to a csv file onto disk

In [ ]:
def tokenize_text(text):
  tokens = [token.text for token in nlp(text)]
  return ' '.join(tokens)

def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'unknown'

In [ ]:
nlp = spacy.load('en_core_sci_md', disable=['parser', 'ner', 'tagger'])

In [ ]:
df = pd.read_csv(path/'raw_dataset.csv')
df.drop_duplicates(inplace=True)
df['note'] = df['category'].str.cat(df['description'], sep='\n')
df['note'] = df['note'].str.cat(df['text'], sep='\n')
df['ethnicity'] = df['ethnicity'].apply(group_eth)
df['scispacy_note'] = df['note'].apply(tokenize_text)
df.head()

In [ ]:
df.to_csv(path/'processed_dataset.csv', index=False)

In [ ]:
full_df = pd.read_csv(path/'full_raw_dataset.csv')

full_df.drop_duplicates(inplace=True)
full_df['note'] = full_df['category'].str.cat(full_df['description'], sep='\n')
full_df['note'] = full_df['note'].str.cat(full_df['text'], sep='\n')
full_df['ethnicity'] = full_df['ethnicity'].apply(group_eth)
full_df['scispacy_note'] = full_df['note'].apply(tokenize_text)
print(full_df.shape)
full_df.head()

In [ ]:
full_df.to_csv(path/'full_processed_dataset.csv', index=False)

## BERT

In [5]:
from utils.splits import set_two_splits

In [6]:
df = pd.read_csv(path/'processed_dataset.csv')
df_split = set_two_splits(df[['note', 'class_label']].copy(), 'val')

In [7]:
df_sample = get_sample(df_split)

In [10]:
df_sample.loc[(df_split['split'] == 'train')].to_csv(path/'bert_processed_train.tsv', sep='\t',
                                                    index_label='index')
df_sample.loc[(df_split['split'] == 'val')].to_csv(path/'bert_processed_val.tsv', sep='\t',
                                                  index_label='index')

In [11]:
df = pd.read_csv(path/'bert_processed_train.tsv', sep='\t')

## Stats

In [5]:
df = pd.read_csv(path/'processed_dataset.csv')

In [26]:
print(f"{(len(df.loc[df['class_label'] == 1])/len(df)):0.3f}")

0.236


In [12]:
from utils.splits import set_two_splits

In [31]:
avg_p = []

for seed in range(127, 227):
  sdf = set_two_splits(df.copy(), name='test', seed=seed)
  test_size = len(sdf.loc[(sdf['split'] == 'test')])
  test_pos = len(sdf.loc[(sdf['split'] == 'test') & (sdf['class_label'] == 1)])
  avg_p.append(test_pos/test_size)  

In [32]:
avg_p = np.array(avg_p)

In [33]:
print(f"Average prevalence = {(avg_p.mean()):0.3f}")

Average prevalence = 0.236


In [34]:
print(f"Std prevalance = {(avg_p.std()):0.3f}")

Std prevalance = 0.004


In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').apply(lambda g: pd.Series(g['subject_id'].nunique()))

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').size()